In [ ]:
!pip install sentence_transformers
!pip install chromadb

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb


class EmbeddingFunction:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

    def __call__(self, input):
        return self.model.encode(input).tolist()


class DB:

    def __init__(self, distance_function, root_path):
        self.ef = EmbeddingFunction()
        self.client = chromadb.PersistentClient(path=root_path)
        self.distance_function = distance_function
        assert distance_function in ["l2", "ip", "cosine"], "Distance function should be 'l2' or 'ip' or 'cosine'"
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

    def add(self, items):
        old_batch = 0
        new_batch = 1000
        while True:
            if new_batch > len(items["fragments"]):
                break
            self.collection.add(
                documents=items["fragments"][old_batch:new_batch],
                metadatas=items["metadata"][old_batch:new_batch],
                ids=items["ids"][old_batch:new_batch])
            old_batch = new_batch
            new_batch += 1000
        self.collection.add(
            documents=items["fragments"][old_batch:],
            metadatas=items["metadata"][old_batch:],
            ids=items["ids"][old_batch:])

    def query(self, query, n_results):
        return self.collection.query(query_embeddings=self.ef(query), n_results=n_results)

    def clear(self):
        self.client.delete_collection("lab5_" + self.distance_function)
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

In [ ]:
from common.utils import split_to_sentences

def split_document(lines, fragment_limit=100):
    sentences = split_to_sentences(lines)
    result = []
    fragment = ""
    length = 0
    for s in sentences:
        fragment += s + " "
        length += len(s.split(" "))
        if length > fragment_limit:
            result.append(fragment)
            fragment = ""
            length = 0
    return result

In [ ]:
import os
import pandas as pd

def split_dataset(dataset_path, fragment_limit=100):
    result_fragments = []
    metadata = []
    result_ids = []
    t = "train.csv"
    filepath = os.path.join(dataset_path, t)
    df = pd.read_csv(filepath, names=['label', 'Title', 'Description'])
    df['text'] = (df['Title'] + '. ' + df['Description'])
    df.drop(columns=['Title', 'Description'], axis=1, inplace=True)
    for index, row in df.iterrows():
      fragments_raw = split_document(row['text'], fragment_limit)
      counter = 0
      for fragment in fragments_raw:
          result_fragments.append(fragment.replace("\n", " "))
          metadata.append({"document": index, "topic": row["label"]})
          result_ids.append(f"{index}_{row['label']}_{counter}")
          counter += 1
    return result_fragments, result_ids, metadata

In [ ]:
data_dir = os.path.realpath("../dataset/raw")

In [ ]:
fragments, ids, metadata = split_dataset(data_dir, fragment_limit=20)

In [ ]:
database_l2 = DB("l2", "./db")
database_ip = DB("ip", "./db")
database_cosine = DB("cosine", "./db")

In [ ]:
database_l2.clear()
database_l2.add({"fragments": fragments, "metadata": metadata, "ids": ids})
database_ip.clear()
database_ip.add({"fragments": fragments, "metadata": metadata, "ids": ids})
database_cosine.clear()
database_cosine.add({"fragments": fragments, "metadata": metadata, "ids": ids})

In [ ]:
database_l2.query("Who loves Christianity?", 5)

In [ ]:
database_ip.query("Who loves Christianity?", 5)

In [ ]:
database_cosine.query("Who loves Christianity?", 5)

In [ ]:
database_l2.query("Кто любит христианство?", 5)

In [ ]:
database_ip.query("Кто любит христианство?", 5)

In [ ]:
database_cosine.query("Кто любит христианство?", 5)